In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import T5Tokenizer
from datasets import Dataset

# 1. Загрузка ВСЕХ данных (English)
print("Загрузка полного датасета...")
train_url = "https://huggingface.co/api/datasets/csebuetnlp/xlsum/parquet/english/train/0.parquet"
df = pd.read_parquet(train_url)

# 2. Инициализация токенизатора для точного подсчета
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

print(f"Всего записей в датасете: {len(df)}")

# 3. Подсчет длин (в символах и токенах)
print("Анализируем длины (это может занять пару минут)...")

# Считаем длину в словах (быстрая оценка)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

# Считаем длину в токенах (точная оценка для модели)
# Берем выборку в 2000 случайных примеров для скорости анализа, 
# либо убери .sample(2000), если хочешь ждать полный расчет
df_sample = df.sample(min(2000, len(df))) 
df_sample['token_count'] = df_sample['text'].apply(lambda x: len(tokenizer.encode(str(x))))

# 4. Вывод статистики
stats = df_sample['token_count'].describe()
print("\n--- Статистика длин текстов (в токенах T5) ---")
print(stats)

# Расчет процента потерь
limit = 512
lost_data = (df_sample['token_count'] > limit).sum() / len(df_sample) * 100
print(f"\nТекстов длиннее {limit} токенов: {lost_data:.2f}%")

# 5. Визуализация
plt.figure(figsize=(12, 6))
sns.histplot(df_sample['token_count'], kde=True, color='skyblue', bins=50)
plt.axvline(limit, color='red', linestyle='--', label=f'Лимит модели ({limit} токенов)')
plt.title('Распределение длин текстов в XLSum (English)')
plt.xlabel('Количество токенов')
plt.ylabel('Количество статей')
plt.legend()
plt.show()